## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-16-10-06-12 +0000,bbc,Nigeria warns against enlisting abroad after ...,https://www.bbc.com/news/articles/cev7e3m04ngo...
1,2026-02-16-10-04-04 +0000,nyt,How Much Will Trump’s Approval Rating Matter i...,https://www.nytimes.com/2026/02/16/upshot/trum...
2,2026-02-16-10-04-00 +0000,nyt,"Iran’s Protests Have Ended, but the Anger and ...",https://www.nytimes.com/2026/02/16/world/middl...
3,2026-02-16-10-03-20 +0000,nyt,‘A Superstar Is From Here’: Pride of Cleveland...,https://www.nytimes.com/2026/02/16/us/laila-ed...
4,2026-02-16-10-03-20 +0000,nypost,Arizona Sheriff Chris Nanos rules out theory N...,https://nypost.com/2026/02/16/us-news/arizona-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2480/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,16
237,new,12
14,iran,8
289,europe,7
35,guthrie,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
86,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...,53
75,2026-02-16-01-20-14 +0000,nypost,Savannah Guthrie tells mom Nancy’s kidnapper ‘...,https://nypost.com/2026/02/15/us-news/savannah...,51
42,2026-02-16-05-39-01 +0000,bbc,Trump's new world order has become real and Eu...,https://www.bbc.com/news/articles/cddn002g6qzo...,46
149,2026-02-15-15-32-07 +0000,bbc,'Trump will be gone in three years': Top Democ...,https://www.bbc.com/news/articles/cjrq2r9y278o...,38
156,2026-02-15-14-04-31 +0000,nyt,In First Public Comments Since Trump’s Racist ...,https://www.nytimes.com/2026/02/15/us/politics...,37


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
86,53,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...
75,51,2026-02-16-01-20-14 +0000,nypost,Savannah Guthrie tells mom Nancy’s kidnapper ‘...,https://nypost.com/2026/02/15/us-news/savannah...
162,33,2026-02-15-13-39-00 +0000,wsj,While the main focus of the Munich Security Co...,https://www.wsj.com/politics/policy/europeans-...
62,31,2026-02-16-02-33-15 +0000,nypost,Radical SoCal group sets next anti-ICE ‘mass m...,https://nypost.com/2026/02/15/us-news/radical-...
168,28,2026-02-15-13-08-14 +0000,nyt,"Gisèle Pelicot, the Woman at the Center of Rap...",https://www.nytimes.com/2026/02/13/magazine/gi...
169,23,2026-02-15-13-00-00 +0000,wsj,Easier access to home loans and modular constr...,https://www.wsj.com/economy/housing/how-congre...
103,23,2026-02-15-22-02-44 +0000,nypost,Calif. lawmaker’s Epstein bombshell outs 4 ran...,https://nypost.com/2026/02/15/us-news/californ...
81,23,2026-02-16-00-33-26 +0000,nypost,Daughters of Irishman held by ICE say father a...,https://nypost.com/2026/02/15/us-news/daughter...
58,23,2026-02-16-02-51-01 +0000,startribune,"Anthony Edwards claims MVP award, leads Stars ...",https://www.startribune.com/scottie-barnes-ot-...
26,21,2026-02-16-09-06-00 +0000,wsj,Oil Broadly Steady Ahead of U.S.-Iran Talks,https://www.wsj.com/finance/commodities-future...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
